In [ ]:
# Common
import os
import keras
import numpy as np

# Data
from keras.preprocessing.image import ImageDataGenerator

# Data Visualization
import plotly.express as px
import matplotlib.pyplot as plt

# Model
from keras.models import Sequential, load_model
from keras.layers import GlobalAvgPool2D as GAP, Dense, Dropout

# Callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Pre-Trained Model
from tensorflow.keras.applications import ResNet50, ResNet50V2, InceptionV3, Xception

In [ ]:
# Class Names
root_path = '/content/drive/MyDrive/shapes/'
class_names = sorted(os.listdir(root_path))
n_classes = len(class_names)
class_names

['circles', 'rectangles', 'stars', 'triangles']

In [ ]:
# Class Distribution
class_dis = [len(os.listdir(root_path + name)) for name in class_names]

# Visualization
fig = px.pie(names=class_names, values=class_dis, title="Class Distribution", hole=0.3)
fig.update_layout({'title':{'x':0.5}})
fig.show()

In [ ]:
# Initialize Generator
gen = ImageDataGenerator(rescale=1/255., validation_split=0.1)

# Load Data
train_ds = gen.flow_from_directory(root_path, class_mode='binary', target_size=(256,256), shuffle=True, batch_size=8, subset='training')
valid_ds = gen.flow_from_directory(root_path, class_mode='binary', target_size=(256,256), shuffle=True, batch_size=8, subset='validation')

Found 721 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [ ]:
def show_images(GRID=[5,5], model=None, size=(20,20), data=train_ds):
    n_rows = GRID[0]
    n_cols = GRID[1]
    n_images = n_cols * n_rows

    i = 1
    plt.figure(figsize=size)
    for images, labels in data:
        id = np.random.randint(len(images))
        image, label = images[id], class_names[int(labels[id])]

        plt.subplot(n_rows, n_cols, i)
        plt.imshow(image)

        if model is None:
            title = f"Class : {label}"
        else:
            pred = class_names[int(np.argmax(model.predict(image[np.newaxis, ...])))]
            title = f"Org : {label}, Pred : {pred}"

        plt.title(title)
        plt.axis('off')

        i+=1
        if i>=(n_images+1):
            break

    plt.tight_layout()
    plt.show()

In [ ]:
show_images()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
BATCH_SIZE = 64
SPE = len(train_ds)//BATCH_SIZE

In [ ]:
# Pre-Trained Model
base_model = InceptionV3(input_shape=(256,256,3), include_top=False)
base_model.trainable = False

# Model Architecture
name = "IV3-Shape-Detector-v2"
model = Sequential([
    base_model,
    GAP(),
    Dense(256, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.4),
    Dense(n_classes, activation='softmax')
], name=name)

# Callbacks
cbs = [EarlyStopping(patience=3, restore_best_weights=True), ModelCheckpoint(name + ".h5", save_best_only=True)]

# Model Compiling
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model Training
model.fit(train_ds, validation_data=valid_ds, callbacks=cbs, epochs=10)

Epoch 1/10
91/91 [==============================] - ETA: 0s - loss: 0.7665 - accuracy: 0.7198

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



91/91 [==============================] - 179s 2s/step - loss: 0.7665 - accuracy: 0.7198 - val_loss: 0.4171 - val_accuracy: 0.8250
Epoch 2/10
91/91 [==============================] - 157s 2s/step - loss: 0.2681 - accuracy: 0.9071 - val_loss: 0.3020 - val_accuracy: 0.8750
Epoch 3/10
91/91 [==============================] - 153s 2s/step - loss: 0.1908 - accuracy: 0.9348 - val_loss: 0.2932 - val_accuracy: 0.9000
Epoch 4/10
91/91 [==============================] - 153s 2s/step - loss: 0.1481 - accuracy: 0.9459 - val_loss: 0.3109 - val_accuracy: 0.9000
Epoch 5/10
91/91 [==============================] - 153s 2s/step - loss: 0.1334 - accuracy: 0.9612 - val_loss: 0.3426 - val_accuracy: 0.8500
Epoch 6/10
91/91 [==============================] - 155s 2s/step - loss: 0.0583 - accuracy: 0.9903 - val_loss: 0.2415 - val_accuracy: 0.9125
Epoch 7/10
91/91 [==============================] - 155s 2s/step - loss: 0.0489 - accuracy: 0.9861 - val_loss: 0.2397 - val_accuracy: 0.9125
Epoch 8/10
91/91 [======

In [ ]:
# Load Model
model_path = '/content/drive/MyDrive/Shape-Detector-v2.h5'
model = load_model(model_path)
model.summary()

Model: "IV3-Shape-Detector-v2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 6, 6, 2048)        21802784  
                                                                 
 global_average_pooling2d_5  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_10 (Dense)            (None, 256)               524544    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_11 (Dense)            (None, 3)                 771       
                                                                 
Total params: 22328099 (85.17 MB)
Trainable params: 525315 (2.00 MB)
Non-trainable params: 21802784 (83.17 MB)


In [ ]:
show_images(model=model, data=valid_ds)


Output hidden; open in https://colab.research.google.com to view.